## Graphics quibs

### Graphics quibs are "live" and interactive

Quibs can be used directly as arguments in most *matplotlib* graphics functions just as if they were standard data variables. Using quibs in graphics functions creates a *graphics quib* - a quib that represents "live" interactive graphics. Indeed, graphics objects created by a graphics quib are bi-directionally linked to their source data: first, any upstream changes are propgated downstream to affect the graphics and, second, user interaction with the data are translated into assignments to the quib arguments and such graphics-driven assignments can propgate to affect upstream quibs through [[inverse assignments]]. 

The following example demonstrates the use of the matplotlib graphics command `bar`, `plot` and `text` with quib arguments:

In [14]:
import pyquibbler as qb
from pyquibbler import iquib, q
qb.override_all()
import numpy as np
import matplotlib.pyplot as plt
%matplotlib tk

In [ ]:
# Figure setup:
plt.ylim([0, 40])
plt.ylabel('v^2')

# Define data quibs:
v = iquib(np.array([5., 3., 1., 2., 4.]));
len_v = q(len, v_sqr)
v_sqr = v ** 2;
v_sqr_mean = np.average(v_sqr);
text_label = q('Average: ()'.format, v_sqr_mean)

# Define graphics quibs:
plt.bar(np.arange(len_v), v_sqr)
range = [-0.5, len_v - 0.5];
plt.xlim(range)
plt.plot(range, v_sqr_mean + [0, 0], 'k:', linewidth=3)
plt.text(np.average(range), v_sqr_mean, text_label, 
    verticalalignment='bottom', horizontalalignment='center', fontsize=16);

The results of these quib graphics commands are *matplotlib* Artist objects that are bi-directionally linked to their source quib data: (1) Upstream changes automatically refresh graphics items and (2) Manual changes to graphic objects are translated backwards into quib assignments. 

### (1) Upstream changes automatically refresh graphics items

In general, when assigning to a quib, the corresponding downstream sub-quibs are invalidated, but are not recalculated until their output value is requested (calculations only occur upon demand). Yet, immediate recalculations do occur when the assignment affects a downstream graphics quib. In this case, recalculation needed for the graphics occur immediately upon new assignments.

For example, setting `v[1] = 3`, will refresh the bar as well as the horizontal average line and the text, all of which depend on `v`.

#### Setters of axis attributes can also work on quibs.
Using `plt.axis`, `plt.xlim`, `plt.ylim` on quibs allows also the axis limits themselves to update dynamically when quib data changes. For example, the statement `plt.ylim([0, max(v_sqr) + 1])`, sets a dynamic y-axis that is automatically updated to reflect changes in the data.

Similarily 

Disable automatic graphics refresh
 

For computationally heavy calculations, we can disable immediate recalculation for graphics quib by setting quib.Central.AutoRefreshGraphics to false (this option can also be set from the quibapp). In this mode, when auto-refresh is disabled, we can use quib.Central.refreshGraphicsNow to manually refresh any un-refreshed graphics quibs. 

(2) Manual changes to graphic objects are translated into quib assignments
We have now seen how upstream changes affect downstream quib graphics. This relationship can also go backwards: changing a graphics object can be translated into assignments to its source quib objects, and from there to any upstream quibs through inverse assignments.
Consider, for example, the presentation of a graphics marker whose X and Y coordinates are specified by two quibs. Moving the graphics marker will change the values of X  and Y. Any quib graphics that depend on X or Y will be immediately refreshed. 
% Figure setup:
figure(4);clf
set(gcf,'Visible','on','toolbar','none')
hold on; box on
axis([-10 10 -10 10])
axis square
% Define quib coordinates:
X = iquib(3);
Y = iquib(4);
% Plot a dot at X, Y (moving the dot will change X and Y):
line('XDATA',X,'YDATA',Y,'marker','o','markersize',20,'markerfacecolor','r')
% plot additional graphics based on X and Y
% (these downstream graphics will change when the marker is moved):
% Text:
text(-9,9,sprintf('X=%4.1f, Y=%4.1f',X,Y),'fontsize',16)
% Rectangle:
plot(X*[-1 1 1 -1 -1],Y*[-1 -1 1 1 -1],'k--','linewidth',1)
% Ellipse:
tt = linspace(0,2*pi,50);
xx = cos(tt)*X;
yy = sin(tt)*Y;
plot(xx,yy,'r-','linewidth',4)
quibdemo_SimpleGraphicsAssignment.gif
Try me: try moving the red marker.
Run this demo,   Open this demo
 
Specifying upon-release and continuous graphics-driven quib assignments
Enabling and specifying the mode of graphic-based quib assignments is indicated simply by specifying the graphic property in capital letters:
Disabling graphics-driven assignment (lower case): Specifying property name in lower case disables graphics-driven assignment. For example, line('xdata',X,'ydata',Y,'marker','o') plots a marker that, while is affected by changes in quibs X or Y, it cannot be dragged to affect X and Y. 
Continuous graphics-driven assignment (UPPER CASE): Specifying property name in UPPER case enables assignment in continuous mode. In this mode, graphics-driven assignments are done continuously as the object is being dragged. As in the example above, line('XDATA',X,'YDATA',Y,'marker','o') plots a marker that is draggable and immediately affects X, Y and any objects that depend on them.
Upon release assignment (mix Upper and Lower case):   Specifying the graphical property in mix lower/upper case enables graphics-driven assignment but only upon release of the dragged object (this is useful when the calculation is too heavy for smooth object dragging). For example, line('XData',X,'YData',Y,'marker','o') yields a draggable marker that affect upstream X and Y quibs only upon mouse release. 
Restricting object dragging horizontally or vertically
By combining properties specified in UPPER and properties specifying with lower case, we can indicate dragging only in X or Y direction. For example, line('XDATA',X,'ydata',Y,'marker','o') enable continuous dragging in the X direction (horizontal); and line('xdata',X,'YData',Y,'marker','o') enable upon-release dragging in the Y direction (vertically). 
 
Example of quib Graphics with specific property-based assignments:
% Figure setup:
figure(5);clf
set(gcf,'Visible','on')
hold on; box on
xlabel('X')
ylabel('Y')
axis([0 10 0 10])
axis square
% Define quib coordinates:
X = iquib(7);
Y = iquib(4);
% Plot vertical and horizontal lines
% use lower/UPPER case to indicate to set the direction for dragging:
line('XDATA',[X X],'ydata',[0 Y],'LineStyle',':','color','k','linewidth',2)
line('xdata',[0 X],'YDATA',[Y Y],'LineStyle',':','color','k','linewidth',2)
% Plot vertical and horizontal sliders
line('XDATA',X,'ydata',0,'marker','^','markerfacecolor','r','markersize',18)
line('xdata',0,'YDATA',Y,'marker','>','markerfacecolor','r','markersize',18)
% Plot a dot at X, Y
% Using CAPS for both XDATA and YDATA indicates draggable in both directions:
line('XDATA',X,'YDATA',Y,'marker','o','markerfacecolor','r','markersize',25)
% Text:
text(X,Y+0.4,sprintf('(%3.1f,%3.1f)',X,Y),'horizontalalignment','center',...
    'verticalalignment','bottom','fontsize',16)
quibdemo_HorzVertDrag.gif
Try me: Check that the sliders can only be dragged horizontally or vertically while the circle can be dragged in both directions. 
Run this demo,   Open this demo
 
Restricting object dragging to a specified path
A more general way to restrict object dragging is by specifying X and Y coordinates that are functionally linked quibs. In this case the object is dragged accroding to the functional constraint. 
Consider the following simple exaple: 
% function of line:
fcn = @(x) 4*x.^2 - x.^3;
% -- X,Y of the point to drag --
X = iquib(3);
Y = fcn(X);
% -- plot --
figure(1); clf
x= 0:0.02:4;
plot(x,fcn(x),'k')
hold on
axis([0 4 0 11])
line('XDATA',X,'YDATA',Y,'marker','o','markerfacecolor','c','markersize',18)
text(X,Y+0.4,sprintf('X=%4.1f,Y=%4.1f',X,Y),...
    'verticalalignment','bottom','horizontalalignment','center','fontsize',16)
quibdemo_DragOnCurve.gif
Applying Matlab's uicontrol or App Designer tools to quib variables generates quib-based GUI 
When Matlab uicontrol or App Designer components are used with quib arguments, user interactions with the graphical controls are automatically translated into assignments into the quib argument (or to more upstream quibs through inverse assignment).
Graphic-driven assignment from uicontrol and App Designer components is enabled when the changed property (typically 'Value') is indicated in UPPER or mix Upper/Lower letter (in both cases only upon-release). 
Consider the following example:
% Defining quib input variables:
n = iquib(120);
nCycles = iquib(30);
% Define downstream quib calculations based on the quib inputs:
phi = linspace(0,2*pi*nCycles,n);
r = linspace(0,10,n);
% Plot the data:
figure(1);clf
axis([-10 10 -10 10])
axis square
hold on
set(gca,'Visible','off')
plot(r.*cos(phi),r.*sin(phi),'r-')
% create uicontrols and labels for the inputs:
uicontrol('Style','slider','Value',nCycles, ...
    'min',0,'max',200,'Position',[10 10 120 20])
uicontrol('Style','text','String',sprintf('Number of cycles %3.0f',nCycles), ...
    'min',0,'max',200,'Position',[10 30 120 20])
uicontrol('Style','slider','Value',n, ...
    'min',0,'max',200,'Position',[10 60 120 20])
uicontrol('Style','text','String',sprintf('Number of Points = %3.0f',n), ...
    'min',0,'max',200,'Position',[10 80 120 20])
quibdemo_uicontrol.gif
Try me: Interacting with the slider controls will be translated into quib assignment and immediately refresh the graphics. 
Run this demo,   Open this demo
 
Quibbler-based GUI applications
Combining uicontrol quibs and draggable quib graphics allows us to quickly and easily build fairly complex GUIs for data analysis. Note that unlike in "regular" programming where we need to specify a callback function for objects or listeners for events, with Quibbler we can easily get interactive functionality without even thinking about callbacks and "what do we need to do if" for every user interaction with controls. 
Example: FFT GUI using quib
% Total time (sec):
T = iquib(100);
% Number of time points (limited to even number between 0 and 1000):
N = iquib(600,'AssignmentTemplate',[0 1000 2]);
% Time vector:
t = linspace(0,T,N);
% Period (sec):
Period = iquib(10);
w = 2*pi/Period;
% Signal (any function of t and w):
Signal = (sin(w*t)>0)*2 - 1;
% Add Noise:
NoideAmp = iquib(0);
Noise = NoideAmp*randn(1,N); % randn is a quib with Cache='off', so return a new result each time its Output is requested
Measure = Signal + Noise;
% Bandwidth:
minF = iquib(0.05);
maxF = iquib(0.6);
% FFT:
Amp = feval(@fft,Measure);
dF = 1/T; % Frequency resolution, 1/TotalTime
F = [0:dF:(N-1)*dF/2 (N)*dF/2:-dF:dF]; % Frequency vector
% Apply band filter
Amp0 = Amp .* (F>=minF & F<=maxF);
% Inverse FFT:
S0 = feval(@ifft,Amp0);
% Figures:
figure(2);clf
set(gcf,'Visible','on','Position',[100 100 720 640])
subplot(3,1,1); hold on; box on
ylim([min(Signal)-0.5-NoideAmp, max(Signal)+0.5+NoideAmp])
xlim([0 T])
xlabel('Time (sec)')
ylabel('Signal')
plot(t,real(S0),'color',[0 0.7 0])
plot(t,Measure,'color',[0.8 0 0])
subplot(3,1,2);hold on; box on
yl = max(abs(Amp))*1.1;
ax = axis([0 max(F) 0 yl]);
xlabel('Frequency (1/sec)')
ylabel('Amplitude')
patch([minF maxF maxF minF],[0 0 yl yl],0,'FaceColor',[1 0.9 0.9])
plot(F,abs(Amp),'k-')
line('XDATA',minF,'YData',0,'Marker','^','MarkerFaceColor','k','MarkerSize',14)
line('XDATA',maxF,'YData',0,'Marker','^','MarkerFaceColor','k','MarkerSize',14)
subplot(3,1,3)
ps = get(gca,'Position');
delete(gca)
ctrl = uipanel(gcf,'Position',ps);
uicontrol(ctrl,'Style','slider','Position',[10 10 200 20],...
    'VALUE',Period,'Min',0,'Max',20)
uicontrol(ctrl,'Style','text','Position',[10 30 200 20],...
    'String',sprintf('Period = %4.1f',Period))
uicontrol(ctrl,'Style','slider','Position',[10 50 200 20],...
    'VALUE',N,'Min',0,'Max',1000)
uicontrol(ctrl,'Style','text','Position',[10 70 200 20],...
    'String',sprintf('Number of Points = %d',N))
uicontrol(ctrl,'Style','slider','Position',[230 10 200 20],...
    'VALUE',T,'Min',0,'Max',200)
uicontrol(ctrl,'Style','text','Position',[230 30 200 20],...
    'String',sprintf('Total Time = %4.0f',T))
uicontrol(ctrl,'Style','slider','Position',[230 50 200 20],...
    'VALUE',NoideAmp,'Min',0,'Max',2)
uicontrol(ctrl,'Style','text','Position',[230 70 200 20],...
    'String',sprintf('Noise Amplitude = %4.1f',NoideAmp))
quibdemo_fft.gif
Try me: Try playing with the control sliders, dragging the low and high band pass (triangle markers), or setting different signal functions. 
Run this demo,   Open this demo
 
Example: Zooming in on Mandelbrot fractal
XY = cell(1,4);
m = cell(1,4);
XY{1} = iquib([-2 1; -1.5 1.5]);
roi0 = [
    287 67 57 60
    146 64 47 60
    137 267 70 57];
figure(1);clf
set(gcf,'Visible','on','Position',[100 100 900 600])
for k = 1:4
    subplot(2,2,k)
    ps = get(gca,'Position');
    m{k} = feval(@mandelbrot,XY{k},500,200);
    imagesc(m{k})
    set(gca,'XTick',[],'YTick',[])
    axis equal
    axis([0 quibsize(m{k},2) 0 quibsize(m{k},1)])
    text(0.03,0.97,num2str(k),'Units','normalized','FontSize',16,'VerticalAlignment','top','HorizontalAlignment','left')
    colormap hot
    if k<4
        roi{k} = iquib(roi0(k,:));
        if exist('images.roi.Rectangle') % check if image processing toolbox exists
            images_roi_Rectangle(gca,'Position',roi{k});
        else
            rectangle(gca,'Position',roi{k});
        end
        XY{k+1} = feval(@roi2xy,roi{k},XY{k},m{k});
    end
end
Try me: Try moving or resizing the Region of Interests (ROIs).
Run this demo,   Open this demo
quibdemo_Mandelbrot.gif
Example: Choosing a region of interest (ROI) of an image
figure(1);clf;
set(gcf,'Visible','on')
% Load and plot an image:
filename = iquib('pipes.jpg');
im = imread(filename);
image(im)
hold on
axis image
set(gca,'Visible','off','ToolBar',[],'Position',[0 0 1 1])
% Define and plot rectangle ROI
% (This rectangle can be moved by the user)
ROI = iquib(int16([100 100 200 150]));
if exist('images.roi.Rectangle') % check if image processing toolbox exists
    images_roi_Rectangle(gca,'POSITION',ROI,'linewidth',3);
else
    rectangle(gca,'POSITION',ROI,'EdgeColor','k','linewidth',3);
end
% Extract and plot the ROI from the main image:
ImCut = feval(@(I,p) I(p(2):p(2)+p(4),p(1):p(1)+p(3),:),im,ROI);
figure(2);clf;
set(gcf,'Visible','on')
subplot(1,2,1)
image(ImCut)
axis equal
axis([0 ROI(3)+1 0 ROI(4)+1])
set(gca,'Visible','off','ToolBar',[])
% Add a rectangle around the extracted ROI:
% (This rectangle too can be moved by the user)
h=rectangle('Position',ROI - [ROI(1:2)-1, 0 0],'linewidth',4);
% Do some downstream analysis on the ROI:
% (This will get automatically updated when the ROI moved)
avgRGB = mean(ImCut,[1 2]);
subplot(1,2,2)
bar(permute(avgRGB,[3 2 1]),'FaceColor',0.7+[0 0 0])
ylim([0 255])
set(gca,'XTickLabel',{'Red','Green','Blue'})
ylabel('Average intensity at the ROI')
quibdemo_defineROI.gif
Run this demo,   Open this demo
Try me: try moving the ROI rectangle either on the main image or around the cut image
 
Using arrayfun/cellfun in conjunction with graphics functions
Applying arrayfun or cellfun in conjunction with quib graphics, allows plotting multiple graphic objects that can be manipulated independently.
The following example illustrates a polynomial fit to a set of points that can be each moved independently by the user. 
% Define a set of X,Y data coordinates:
x = iquib(1:10);
y = iquib(100 - (1:10).^2+5*randn(1,10));
% Define polynomial degree:
n = iquib(2);
% Use AssignmentTemplate to specify minimal maximal
% and step values for the polynomial degree:
n.AssignmentTemplate = [0 5 1];
figure(1); clf; hold on; box on
set(gcf, 'toolbar', 'none')
axis([0 11 0 110])
set(gca,'position',[0.15 0.2 0.7 0.75])
% Use arrayfun to plot each data-point.
ar = arrayfun(@line,"XDATA",x,"YDATA",y, ...
    "marker",'o',"markersize",12,"markerfacecolor",'y');
ar.Out; % to invoke calculating and ploting immediately.
% Calculate and plot the polynomial fit:
pf = polyfit(x,y,n);
x0 = linspace(min(x),max(x),30);
y0 = polyval(pf,x0);
plot(x0,y0,'k-')
% Add a slider for the polynomial degree:
uicontrol('Style','slider','VALUE',n,'min',0,'max',5,'SliderStep',[0.2 0.2],...
    'units','normalized', 'Position',[0.15 0.05 0.2 0.05])
uicontrol('Style','text', 'String', sprintf('Polynom degree = %d',n),...
    'units','normalized', 'Position',[0.15 0.1 0.2 0.05])
quibdemo_polyfit.gif
Try me: Moving any of the points, or moving the degree slider, will result in immediate recalculation of the polynomial fit.
Run this demo,   Open this demo
 
See also
quib.IsGraphics
Examples
Quibbler table of content
 
 
 
Support functions for Mandelbrot demo above:
function xy2 = roi2xy(roi,xy1,img)
sz = size(img);
r = reshape(roi,2,2); % [x dx; y, dy]
r(:,2) = r(:,1)+r(:,2);
r = r ./ [sz(2);sz(1)];
xy2 = xy1(:,1) + (xy1(:,2)-xy1(:,1)).*r;
end
function m = mandelbrot(xyRange,n,niter)
if nargin<2
    n = 1000;
end
if nargin<3
    niter = 30;
end
xRange = xyRange(1,:);
yRange = xyRange(2,:);
dy = diff(yRange);
dx = diff(xRange);
d = max(dy,dx)/n;
[x,y] = meshgrid(xRange(1):d:xRange(2), yRange(1):d:yRange(2));
c = x + 1i * y;
z = zeros(size(c));
m = zeros(size(c));
for ii = 1:niter
    z = z.^2 + c;
    m(abs(z) > 2 & m == 0) = niter - ii;
end
end